In [4]:
# Setting up the environments
import numpy as np
import matplotlib as ml
import matplotlib.pyplot as plt
import ioeddy as io 
import matplotlib.gridspec as gridspec
import pandas as pd
import struct as st
import seaborn as sns
import cmocean
import palettable
import time
import csv
import ioeddy as io
from scipy.io import FortranFile

In [13]:
# Read Grid
nx, index, x, xe, xc = io.readgrid('/home/sheel/Work/projects/spod_re5e4/grid/frinf/x3_grid.in')
nr, index, r, re, rc = io.readgrid('/home/sheel/Work/projects/spod_re5e4/grid/frinf/x1_grid.in')
print("Number of grid points in x direction", nx)
print("Number of grid points in r direction", nr)

Number of grid points in x direction 4609
Number of grid points in r direction 365


In [14]:
# Map the grid to actual location of the data files
kmin = 298
kmax = 302

NKRR = np.int(kmax - kmin + 1 + 5*np.floor((4610-kmax)/100))
print('NKRR', NKRR) 

xc_data_file = np.zeros([NKRR],dtype=float)
xc_data_file[0:kmax-kmin+1] = xc[kmin-1:kmax] # Storing the disk location

mk = (kmax-kmin+1)   
nk = 1

for i in range(0,np.int(np.floor((4610-kmax)/100))):
    i_actual = kmax + nk*100 - 1
    xc_data_file[mk]   = xc[i_actual-2]  
    xc_data_file[mk+1] = xc[i_actual-1]  
    xc_data_file[mk+2] = xc[i_actual]  
    xc_data_file[mk+3] = xc[i_actual+1] 
    xc_data_file[mk+4] = xc[i_actual+2]  
    mk = mk + 5
    nk = nk + 1
    
#print(xc_data_file)

rc_data_file = rc[0:nr-10+1]

NKRR 220


In [15]:
# Extracting array location at a given x/D
v = np.array([5, 10, 15, 60, 65, 70, 100])
idx = np.zeros((7))
size_v = np.shape(v)
for i in range(0, size_v[0]):
    idx[i] =  np.int((np.abs(xc_data_file - v[i])).argmin())
idx = idx.astype(int)
print(xc_data_file[idx]) 

[ 5.23045932  9.48706881 15.02007802 60.87916779 64.98611302 69.05460349
 98.41029527]


In [25]:
print(idx[6])

189


In [71]:
# Read 5p data file
def read5p2(filename):
        f = open(filename,'rb')
        _,i,j,k,jp,_ = st.unpack('i'*6,f.read(24))
        k = 220
        i = i - 10
        print('i',i)
        
        data=np.zeros([i,j,k],dtype=float)

        for kk in range (0,k):
    
                dummy1 = st.unpack('i',f.read(4))    
                data[:,:,kk]=np.reshape(st.unpack('d'*i*j,f.read(8*i*j)),(i,j),order='F')
                dummy2 = st.unpack('i',f.read(4))
    
                if dummy1 != dummy2:
                        print('Error reading',kk)
                        break
        
        _,it,_,_,time,_,_,dt,grav,_=st.unpack('iiiidiiddi',f.read(4*7+3*8))
        
        return i,j,k,it,time,dt,grav,data
        print('Read the data file')

In [72]:
nstart = 1892600
nend   = 2613200
stride = 100
ntheta = 258
tot_size = np.int((nend-nstart)/stride + 1)
path_up = '/p/work/snidhan/spod_re5e4/frinf/DATA_FILES/up_datafiles/'
path_vp = '/p/work/snidhan/spod_re5e4/frinf/DATA_FILES/vp_datafiles/'
path_wp = '/p/work/snidhan/spod_re5e4/frinf/DATA_FILES/wp_datafiles/'

file_up = 'up_'
file_vp = 'up_'
file_wp = 'up_'

count = 0
time_array = np.zeros([tot_size],dtype='float64')
data_idx = np.zeros([356,258,size_v[0]],dtype='float64')

for step in range(nstart,nend+100,100):
    
    print('Entering this loop')
    
    i_pad = str(step).zfill(8)
    
    filename = path_up + file_up +  i_pad + '.res'
    i,j,k,it,time,dt,grav,data = read5p2(filename)
    print(filename) 
    
    time_array[count] = time
    
    for i in range(0,size_v[0]):
        filename1 = path_up + file_up + i_pad + '_'+ str(idx[i]) + '.res'
        data_f = data_idx[:,:,idx[i]].reshape(nr-10+1,ntheta,order='F')
        data_f.T.tofile(filename1)

    filename = path_vp + file_vp +  i_pad + '.res'
    i,j,k,it,time,dt,grav,data = read5p2(filename)
    print(filename) 
    
    for i in range(0,size_v[0]):
        filename1 = path_vp + file_vp + i_pad + '_'+ str(idx[i]) + '.res'
        data_f = data_idx[:,:,idx[i]].reshape(nr-10+1,ntheta,order='F')
        data_f.T.tofile(filename1)

    filename = path_wp + file_wp +  i_pad + '.res'
    i,j,k,it,time,dt,grav,data = read5p2(filename)
    print(filename) 
    
    for i in range(0,size_v[0]):
        filename1 = path_wp + file_wp + i_pad + '_'+ str(idx[i]) + '.res'
        data_f = data_idx[:,:,idx[i]].reshape(nr-10+1,ntheta,order='F')
        data_f.T.tofile(filename1)
        
    print('Written ', step)
    
    file = 'time_stamp.txt'
    filename = path + file
    
    f = open(filename, "w")
    
    for x in range(0,tot_size):
        print(x)
        f.write(str(time_array[x]) + "\n")
    f.close() 

Entering this loop


FileNotFoundError: [Errno 2] No such file or directory: '/p/work/snidhan/spod_re5e4/frinf/DATA_FILES/up_datafiles/up_01892600.res'